## Week 4, Lab 1: Predicting Left-Handedness from Psychological Factors
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

We'll walk through a full data science problem in this lab. 

In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

---
## Step 1: Define The Problem.

You're currently a data scientist working at a university. A professor of psychology is attempting to study the relationship between personalities and left-handedness. They have tasked you with gathering evidence so that they may publish.

Specifically, the professor says "I need to prove that left-handedness is caused by some personality trait. Go find that personality trait and the data to back it up."

As a data scientist, you know that any real data science problem must be **specific** and **conclusively answerable**. For example:
- Bad data science problem: "What is the link between obesity and blood pressure?"
    - This is vague and is not conclusively answerable. That is, two people might look at the conclusion and one may say "Sure, the problem has been answered!" and the other may say "The problem has not yet been answered."
- Good data science problem: "Does an association exist between obesity and blood pressure?"
    - This is more specific and is conclusively answerable. The problem specifically is asking for a "Yes" or "No" answer. Based on that, two independent people should both be able to say either "Yes, the problem has been answered" or "No, the problem has not yet been answered."
- Excellent data science problem: "As obesity increases, how does blood pressure change?"
    - This is very specific and is conclusively answerable. The problem specifically seeks to understand the effect of one variable on the other.

### 1. In the context of the left-handedness and personality example, what are three specific and conclusively answerable problems that you could answer using data science? 

> You might find it helpful to check out the codebook in the repo for some inspiration.

**Answer:**
1. Are left handers more likely to be non-heterosexual? --> This is our main problem statement
2. Are left handers more likely to be female than male?
3. Are left handers more likely to hate shopping? (Q33)

---
## Step 2: Obtain the data.

### 2. Read in the file titled "data.csv."
> Hint: Despite being saved as a .csv file, you won't be able to simply `pd.read_csv()` this data!

In [2]:
df = pd.read_csv('data.csv', delimiter='\t')
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,country,fromgoogle,engnat,age,education,gender,orientation,race,religion,hand
0,4,1,5,1,5,1,5,1,4,1,...,US,2,1,22,3,1,1,3,2,3
1,1,5,1,4,2,5,5,4,1,5,...,CA,2,1,14,1,2,2,6,1,1
2,1,2,1,1,5,4,3,2,1,4,...,NL,2,2,30,4,1,1,1,1,2
3,1,4,1,5,1,4,5,4,3,5,...,US,2,1,18,2,2,5,3,2,2
4,5,1,5,1,5,1,5,1,3,1,...,US,2,1,22,3,1,1,3,2,3


### 3. Suppose that, instead of us giving you this data in a file, you were actually conducting a survey to gather this data yourself. From an ethics/privacy point of view, what are three things you might consider when attempting to gather this data?
> When working with sensitive data like sexual orientation or gender identity, we need to consider how this data could be used if it fell into the wrong hands!

**Answer:** Race, religion and orientation. If these variables are not relevant to our purpose in conducting the study, it may not be necessary to have these questions.

---
## Step 3: Explore the data.

### 4. Conduct exploratory data analysis on this dataset.
> If you haven't already, be sure to check out the codebook in the repo, as that will help in your EDA process.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4184 entries, 0 to 4183
Data columns (total 56 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Q1           4184 non-null   int64 
 1   Q2           4184 non-null   int64 
 2   Q3           4184 non-null   int64 
 3   Q4           4184 non-null   int64 
 4   Q5           4184 non-null   int64 
 5   Q6           4184 non-null   int64 
 6   Q7           4184 non-null   int64 
 7   Q8           4184 non-null   int64 
 8   Q9           4184 non-null   int64 
 9   Q10          4184 non-null   int64 
 10  Q11          4184 non-null   int64 
 11  Q12          4184 non-null   int64 
 12  Q13          4184 non-null   int64 
 13  Q14          4184 non-null   int64 
 14  Q15          4184 non-null   int64 
 15  Q16          4184 non-null   int64 
 16  Q17          4184 non-null   int64 
 17  Q18          4184 non-null   int64 
 18  Q19          4184 non-null   int64 
 19  Q20          4184 non-null 

In [4]:
df.race.unique()

array([3, 6, 1, 2, 7, 0, 4, 5])

### Data Cleaning:

1. Drop outlier age (above 400). 2 observations dropped

In [5]:
df.drop(df.loc[df['age'] > 400].index, inplace=True)
df.shape

(4182, 56)

2. Drop hand = 0. 10 observations dropped

In [6]:
df.drop(df.loc[df['hand'] == 0].index, inplace=True)
df.shape

(4172, 56)

3. Drop column 'country' since not useful for problem statement.

In [7]:
df.drop(columns='country', inplace=True)
df.shape

(4172, 55)

4. Drop where testelepase is less than 1.5 minutes as it is unlikely to be humanely possible to do the 56 questions within this time frame. 11 observations dropped.

In [8]:
df.drop(df.loc[df['testelapse'] < 90, 'testelapse'].index, inplace=True)
df.shape

(4161, 55)

5. Drop where testelapse is more than 60 minutes as it is more likely to be unfocused. 33 observations dropped.

In [9]:
df.drop(df.loc[df['testelapse'] > 3600, 'testelapse'].index, inplace=True)
df.shape

(4128, 55)

6. Drop where introelapse is more than 1 hour as it is more likely to have unfocused answers. 33 observations dropped.

In [10]:
df.drop(df.loc[df['introelapse'] > 3600, 'introelapse'].index, inplace=True)
df.shape

(4095, 55)

---
## Step 4: Model the data.

### 5. Suppose I wanted to use Q1 - Q44 to predict whether or not the person is left-handed. Would this be a classification or regression problem? Why?

**Answer:** Classification, since the questions produce a categorical outcome (left or right or ambi) instead of a continous outcome.

### 6. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed based on their responses to Q1 - Q44. Before doing that, however, you remember that it is often a good idea to standardize your variables. In general, why would we standardize our variables? Give an example of when we would standardize our variables.

**Answer:** K-nearest neighbors predictions are made by measuring the distance of the points. Standardizing variables are done to avoid the effect of magnitude of certain features (X columns, e.g. width in inch and length in miles).

### 7. Give an example of when we might not standardize our variables.

**Answer:** We do not need to standardize our variables if it does not require measuring by distance or if the variables have the same units.

### 8. Based on your answers to 6 and 7, do you think we should standardize our predictor variables in this case? Why or why not?

**Answer:** Since we are only basing the responses based on Q1-Q44, we do not need to standardize because they are all on the same range of values (1-5)

### 9. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed. What munging/cleaning do we need to do to our $y$ variable in order to explicitly answer this question? Do it.

**Answer:** As of now, we have no y variables. We need to input 'hand' column as only 0 and 1. We include ambidextrous as left handed instead of right, since they can use their left hand.

In [11]:
df['left'] = [1 if i in [2, 3] else 0 for i in df['hand']]
df['left'].sum()

624

In [12]:
df.loc[(df['hand'] == 2) | (df['hand'] == 3),'hand'].count()

624

### 10. The professor for whom you work suggests that you set $k = 4$. In this specific case, why might this be a bad idea?

**Answer:** When k is an even number, the results may be tied (2-2, 5-5) and thus will not produce an answer relevant to our question.

### 11. Let's *(finally)* use $k$-nearest neighbors to predict whether or not a person is left-handed!

> Be sure to create a train/test split with your data!

> Create four separate models, one with $k = 3$, one with $k = 5$, one with $k = 15$, and one with $k = 25$.

> Instantiate and fit your models.

In [13]:
df.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31',
       'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41',
       'Q42', 'Q43', 'Q44', 'introelapse', 'testelapse', 'fromgoogle',
       'engnat', 'age', 'education', 'gender', 'orientation', 'race',
       'religion', 'hand', 'left'],
      dtype='object')

In [14]:
X = df.drop(columns=['introelapse', 'testelapse', 'fromgoogle', 'engnat', 'age', 'education', 'gender',
                     'religion', 'hand', 'left', 'orientation', 'race'])
y = df['left']
print(type(X))
print(type(y))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    stratify=y
                                                   )

knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_5 = KNeighborsClassifier(n_neighbors=5)
knn_15 = KNeighborsClassifier(n_neighbors=15)
knn_25 = KNeighborsClassifier(n_neighbors=25)

knn_3.fit(X_train, y_train)
knn_5.fit(X_train, y_train)
knn_15.fit(X_train, y_train)
knn_25.fit(X_train, y_train)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


KNeighborsClassifier(n_neighbors=25)

Being good data scientists, we know that we might not run just one type of model. We might run many different models and see which is best.

### 12. We want to use logistic regression to predict whether or not a person is left-handed. Before we do that, let's check the [documentation for logistic regression in sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Is there default regularization? If so, what is it? If not, how do you know?

**Answer:** Default is L2 penalty


### 13. We want to use logistic regression to predict whether or not a person is left-handed. Before we do that, should we standardize our features?

**Answer:** Yes we should, since we are applying a penalty (L2 regularization).

### 14. Let's use logistic regression to predict whether or not the person is left-handed.


> Be sure to use the same train/test split with your data as with your $k$-NN model above!

> Create four separate models, one with LASSO and $\alpha = 1$, one with LASSO and $\alpha = 10$, one with Ridge and $\alpha = 1$, and one with Ridge and $\alpha = 10$. *(Hint: Be careful with how you specify $\alpha$ in your model!)*

> Instantiate and fit your models.

In [15]:
logreg_lasso_1 = LogisticRegression(penalty='l1', C=1, solver='liblinear')
logreg_lasso_1.fit(X_train, y_train)

logreg_lasso_10 = LogisticRegression(penalty='l1', C=10, solver='liblinear')
logreg_lasso_10.fit(X_train, y_train)

logreg_ridge_1 = LogisticRegression(penalty='l2', C=1, solver='liblinear')
logreg_ridge_1.fit(X_train, y_train)

logreg_ridge_10 = LogisticRegression(penalty='l2', C=10, solver='liblinear')
logreg_ridge_10.fit(X_train, y_train)


LogisticRegression(C=10, solver='liblinear')

---
## Step 5: Evaluate the model(s).

### 15. Before calculating any score on your data, take a step back. Think about your $X$ variable and your $Y$ variable. Do you think your $X$ variables will do a good job of predicting your $Y$ variable? Why or why not? What impact do you think this will have on your scores?

Answer:

### 16. Using accuracy as your metric, evaluate all eight of your models on both the training and testing sets. Put your scores below. (If you want to be fancy and generate a table in Markdown, there's a [Markdown table generator site linked here](https://www.tablesgenerator.com/markdown_tables#).)
- Note: Your answers here might look a little weird. You didn't do anything wrong; that's to be expected!

**Answer:**

| Type   | n_neighbors | Train Score  |    Test Score    |-| Type   | Ridge/Lasso | alpha  | Train Score  | Test Score|
|--------|-------------|--------|--------|-|-------|-------------|--------|--------|---|
| KNN    | 3           | 0.805  | 0.7929 || Logreg | Lasso       | 1      | 0.8476 |0.8438|
|        | 5           | 0.8294 | 0.8242 ||        |             | 10     | 0.8476 |0.8438|
|        | 15          | 0.8466 | 0.8467 ||        | Ridge       | 1      | 0.8476 |0.8438|
|        | 25          | 0.8476 | 0.8477 ||        |             | 10     | 0.8476 |0.8438|


In [16]:
print(f'KNN 3  Train:  {round(cross_val_score(knn_3, X_train, y_train).mean(), 4)}, \
     Test:   {round(cross_val_score(knn_3, X_test, y_test).mean(), 4)}')

print(f'KNN 5  Train:  {round(cross_val_score(knn_5, X_train, y_train).mean(), 4)}, \
    Test:   {round(cross_val_score(knn_5, X_test, y_test).mean(), 4)}')

print(f'KNN 15 Train:  {round(cross_val_score(knn_15, X_train, y_train).mean(), 4)}, \
    Test:   {round(cross_val_score(knn_15, X_test, y_test).mean(), 4)}')

print(f'KNN 25 Train:  {round(cross_val_score(knn_25, X_train, y_train).mean(), 4)}, \
    Test:   {round(cross_val_score(knn_25, X_test, y_test).mean(), 4)}')

KNN 3  Train:  0.8053,      Test:   0.7822
KNN 5  Train:  0.8336,     Test:   0.8242
KNN 15 Train:  0.8473,     Test:   0.8447
KNN 25 Train:  0.8476,     Test:   0.8477


In [17]:
print(f'Logreg Lasso 1  Train: {round(cross_val_score(logreg_lasso_1, X_train, y_train).mean(), 4)}, \
      Test: {round(cross_val_score(logreg_lasso_1, X_test, y_test).mean(), 4)}')

print(f'Logreg Lasso 10 Train: {round(cross_val_score(logreg_lasso_10, X_train, y_train).mean(), 4)}, \
      Test: {round(cross_val_score(logreg_lasso_10, X_test, y_test).mean(), 4)}')

print(f'Logreg Ridge 1  Train: {round(cross_val_score(logreg_ridge_1, X_train, y_train).mean(), 4)}, \
      Test: {round(cross_val_score(logreg_lasso_1, X_test, y_test).mean(), 4)}')

print(f'Logreg Ridge 10 Train: {round(cross_val_score(logreg_ridge_10, X_train, y_train).mean(), 4)}, \
      Test: {round(cross_val_score(logreg_lasso_1, X_test, y_test).mean(), 4)}')

Logreg Lasso 1  Train: 0.8476,       Test: 0.8438
Logreg Lasso 10 Train: 0.8476,       Test: 0.8438
Logreg Ridge 1  Train: 0.8476,       Test: 0.8438
Logreg Ridge 10 Train: 0.8476,       Test: 0.8438


### 17. In which of your $k$-NN models is there evidence of overfitting? How do you know?

**Answer:** Overfitting is seen when training score is better than testing score. We see evidencee of overfitting where k=3 and k=5

### 18. Broadly speaking, how does the value of $k$ in $k$-NN affect the bias-variance tradeoff? (i.e. As $k$ increases, how are bias and variance affected?)

**Answer:** As k increases, bias increases while variance decreases

### 19. If you have a $k$-NN model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

**Answer:**
1. Increase k
2. Decrease variables
3. Increase regularization (alpha penalty)

### 20. In which of your logistic regression models is there evidence of overfitting? How do you know?

**Answer:** All models are overfit. All training scores are higher than testing scores

### 21. Broadly speaking, how does the value of $C$ in logistic regression affect the bias-variance tradeoff? (i.e. As $C$ increases, how are bias and variance affected?)

**Answer:** C is alpha. When C increases, regularization increases which results in a lower variance but higher bias.

### 22. For your logistic regression models, play around with the regularization hyperparameter, $C$. As you vary $C$, what happens to the fit and coefficients in the model? What do you think this means in the context of this specific problem?

**Answer:** There is no change in the model, which means that regularization may not be required or that a small amount of regulatization is enough since it provides the same results as more regularization.

### 23. If you have a logistic regression model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

**Answer:**
1. Increase regularization by increasing C (alpha).
2. Decrease the number of variables used.
3. Increase k to decrease variance.

---
## Step 6: Answer the problem.

### 24. Suppose you want to understand which psychological features are most important in determining left-handedness. Would you rather use $k$-NN or logistic regression? Why?

**Answer:** Logistic regression. Using logreg allows us to find the rate of probability increase when some factors increase by 1 (For instance, an answer changes from 4 to 5)

### 25. Select your logistic regression model that utilized LASSO regularization with $\alpha = 1$. Interpret the coefficient for `Q1`.

**Answer:**

Q1	I have studied how to win at gambling.

1 increase in Q1 answer results in a 1.01% higher likelihood for the person to be left handed.

In [21]:
np.exp(0.010072)

1.010122893314311

In [19]:
pd.DataFrame(logreg_lasso_1.coef_[0], index=X.columns)[:5]

,0
Q1,0.010072
Q2,0.013736
Q3,0.026402
Q4,0.000000
Q5,0.041097


### 26. If you have to select one model overall to be your *best* model, which model would you select? Why?
- Usually in the "real world," you'll fit many types of models but ultimately need to pick only one! (For example, a client may not understand what it means to have multiple models, or if you're using an algorithm to make a decision, it's probably pretty challenging to use two or more algorithms simultaneously.) It's not always an easy choice, but you'll have to make it soon enough. Pick a model and defend why you picked this model!

**Answer:** KNN model with k=5. It has the best bias to variance tradeoff as compared to k= 3, 15 or 25. Additionally, the logreg model provides the same value which suggest that the questions may not have any impact on the outcome.

### 27. Circle back to the three specific and conclusively answerable questions you came up with in Q1. Answer one of these for the professor based on the model you selected!

**Answer:** If a person's answer for Q33 increases by 1, the likelihood of them being left-handed increases by 3.04%

In [26]:
display(pd.DataFrame(logreg_lasso_1.coef_[0], index=X.columns)[32:33])
display(np.exp(0.030015)-1)

,0
Q33,0.030015


0.030469990887452836

### BONUS:
Looking for more to do? Probably not - you're busy! But if you want to, consider exploring the following. (They could make for a blog post!)
- Create a visual plot comparing training and test metrics for various values of $k$ and various regularization schemes in logistic regression.
- Rather than just evaluating models based on accuracy, consider using sensitivity, specificity, etc.
- In the context of predicting left-handedness, why are unbalanced classes concerning? If you were to re-do this process given those concerns, what changes might you make?
- Fit and evaluate a generalized linear model other than logistic regression (e.g. Poisson regression).
- Suppose this data were in a `SQL` database named `data` and a table named `inventory`. What `SQL` query would return the count of people who were right-handed, left-handed, both, or missing with their class labels of 1, 2, 3, and 0, respectively? (You can assume you've already logged into the database.)